## Importando dependências

In [ ]:
!python -m pip install scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importando dados

In [ ]:
df_main = pd.read_csv('data/IM_230626_semNP.csv')

## Carteira comercial

A métrica utilizada será a taxa de inadimplência, calculada pela carteira de direitos de aquisição inadimplentes dividido pelo patrimônio líquido

In [ ]:
df_main = df_main.dropna(subset=["Patrimonio_Liquido"])
inadimplentes = df_main["Carteira_Direitos_Aquisicao_Inadimplentes"]
patrimonio_liquido = df_main["Patrimonio_Liquido"]
taxa_inadimplencia = inadimplentes / patrimonio_liquido

df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)


In [ ]:
fundos = df_main[
    (df_main['taxa_inadimplencia_series'] != 0) &
    df_main['taxa_inadimplencia_series'].notna()
]

list(fundos['Nome_Fundo'].unique())

selecionou-se os fundos cuja taxa de inadimplência fosse válida, ou seja, apenas aqueles que a taxa de inadimplência fosse diferente de 0 e NaN

In [ ]:
carteiras = [
    'Carteira',
    'Carteira_Industrial',
    'Carteira_Mercado_Imobiliario',
    'Carteira_Comercial_Total',
    'Carteira_Comercial',
    'Carteira_Comercial_Varejo',
    'Carteira_Arrendamento_Mercantil',
    'Carteira_Servicos_Total',
    'Carteira_Servicos',
    'Carteira_Servicos_Publicos',
    'Carteira_Servicos_Educacionais',
    'Carteira_Entretenimento',
    'Carteira_Agronegocio',
    'Carteira_Financeiro',
    'Carteira_Credito_Pessoal_Consignado',
    'Carteira_Credito_Corporativo',
    'Carteira_Middle_Market',
    'Carteira_Veiculos',
    'Carteira_Imobiliaria_Empresarial',
    'Carteira_Imobiliaria_Residencial',
    'Carteira_Outros_Financeiro',
    'Carteira_Cartao_Credito',
    'Carteira_Factoring',
    'Carteira_Factoring_Pessoal',
    'Carteira_Factoring_Corporativo',
    'Carteira_Setor_Publico',
    'Carteira_Precatorios',
    'Carteira_Creditos_Tributarios',
    'Carteira_Royalties',
    'Carteira_Outros_Setor_Publico',
    'Carteira_Acoes_Judiciais',
    'Carteira_Propriedade_Intelectual',
]

soma_carteiras = df_main.groupby('taxa_inadimplencia_series')[carteiras].sum()
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=soma_carteiras, palette='magma')
plt.title('Taxas de Inadimplência por Carteira')
plt.ylabel('Taxa total de inadimplência')
plt.xlabel('Carteira')
plt.xticks(rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()

plt.show()

O gráfico foi plotado com todos os tipos de carteira, avaliando a taxa de inadimplência de cada segmento.

In [ ]:
soma_carteiras.head(5)

## K-means

O K-means é um algoritmo de agrupamento de dados que divide um conjunto de pontos em grupos (clusters) com base em suas características similares, buscando minimizar a variância dentro de cada grupo.

In [ ]:
from sklearn.cluster import KMeans

carteira_comercial = 'Carteira_Comercial_Total'

fundos = df_main[df_main[carteira_comercial].notna()]

X = fundos[[carteira_comercial]]

kmeans = KMeans(n_clusters=3)
fundos['grupo'] = kmeans.fit_predict(X)

cores = ['#FFA500', '#00CED1', '#00008B']
fig, ax = plt.subplots(figsize=(12, 8))

for grupo in range(3):
    grupo_df = fundos[fundos['grupo'] == grupo]
    ax.scatter(grupo_df.index, grupo_df[carteira_comercial],
               color=cores[grupo], label=f'Cluster {grupo}', s=100, alpha=0.7)

ax.set_title(f'{carteira_comercial}', fontsize=16)
ax.set_xlabel('.', fontsize=14)
ax.set_ylabel('Inadimplência', fontsize=14)
ax.set_xticks([])
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)

ax.set_facecolor('white')

plt.tight_layout()
plt.show()

In [ ]:
cluster1 = fundos[fundos['grupo'] == 1]

cluster1 = cluster1.sort_values(by='taxa_inadimplencia_series', ascending=False)

fundos_inadimplencia = cluster1.head(10)

for index, row in fundos_inadimplencia.iterrows():
    print(f'Nome do Fundo: {row["Nome_Fundo"]}')
    print(f'CNPJ do Fundo: {row["CNPJ"]}')
    print(f'Taxa de Inadimplência: {row["taxa_inadimplencia_series"]}')
    print(f'Data de competência: {row["Data_Competencia"]}')
    print('-' * 30)


Mostra as maiores taxas de inadimplência do cluster 1, ou seja do cluster que agrupa as maiores taxas de inadimplência dos dados.

#### Silhouette Score
O Silhouette Score avalia a coesão interna dos clusters e a separação entre os clusters. Ele varia de -1 a 1, com valores mais próximos de 1 indicando um bom agrupamento. Um valor negativo sugere que os pontos podem ter sido atribuídos ao cluster errado.

In [ ]:
from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(X, fundos['grupo'])
f'Coeficiente de Silhueta: {silhouette_avg}'

#### Utilização do K-Means como Primeiro Modelo Candidato para o Problema

Na busca por uma solução eficiente para o problema em questão, a escolha inicial recaiu sobre o algoritmo K-Means. O K-Means é um método de clustering (agrupamento) amplamente utilizado na análise de dados e aprendizado de máquina. Sua aplicação visa agrupar dados em clusters, onde cada cluster contém pontos de dados que são mais semelhantes entre si do que com pontos de dados de outros clusters.

#### Justificativa para a Definição do K do Modelo

A determinação do número de clusters (K) é crucial para o sucesso do K-Means. Nesse contexto, foi empregada a métrica do coeficiente de silhueta (silhouette score) para encontrar o valor mais adequado de K. O coeficiente de silhueta mede a similaridade média entre um objeto e seu cluster em relação aos outros clusters, variando de -1 (representando uma alocação de cluster inadequada) a 1 (indicação de uma alocação ideal).

No teste realizado, o coeficiente de silhueta obteve um valor considerável de 0.9396926895140495. Esse resultado sugere que os clusters encontrados pelo K-Means são bem definidos e que a alocação de cada ponto de dados ao seu cluster é apropriada. Portanto, o valor de K utilizado nesse modelo é substancialmente apropriado para a tarefa, proporcionando uma segmentação eficaz dos dados.

#### DB Scan

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) é um algoritmo de clustering que agrupa pontos de dados próximos com base na densidade local. Ele não requer a pré-especificação do número de clusters e pode identificar clusters de diferentes tamanhos e formas. O algoritmo distingue entre pontos centrais (que têm vizinhos próximos o suficiente), pontos de borda (que estão próximos de pontos centrais) e pontos de ruído (que não pertencem a nenhum cluster). É útil para análise de dados espaciais e requer a definição adequada de parâmetros, como MinPts (número mínimo de pontos) e ε (distância máxima entre pontos) para funcionar bem em um conjunto de dados específico.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

#### Comercial x Serviços


In [ ]:
df_cleaned = df_main.dropna(subset=["Patrimonio_Liquido"])

inadimplentes = df_cleaned["Carteira_Direitos_Aquisicao_Inadimplentes"]

patrimonio_liquido = df_cleaned["Patrimonio_Liquido"]

taxa_inadimplencia = inadimplentes / patrimonio_liquido

taxa_inadimplencia

df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)

In [ ]:
fundos_inadimplentes = df_main[
    (df_main['taxa_inadimplencia_series'] != 0) &
    df_main['taxa_inadimplencia_series'].notna()
]

list(fundos_inadimplentes['Nome_Fundo'].unique())

In [ ]:
carteiras = [
    'Carteira_Servicos',
    'Carteira_Comercial',
]

summed_data = df_main.groupby('taxa_inadimplencia_series')[carteiras].sum()
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=summed_data, palette='viridis')
plt.title('Sum of Default Rates by Portfolio')
plt.xlabel('Total Default Rate')
plt.ylabel('Portfolio')
plt.xticks(rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
summed_data.head(5)
X = summed_data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

# Selecione as colunas relevantes para o DBSCAN
X = df_main[['Carteira_Servicos', 'Carteira_Comercial']]

# Remova os valores NaN do DataFrame
X.dropna(inplace=True)

# Amostragem aleatória de 10% dos dados
sampled_data = X.sample(frac=0.1, random_state=42)

# Aplique o StandardScaler aos dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(sampled_data)

# Crie clusters usando DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Ajuste os hiperparâmetros conforme necessário
labels = dbscan.fit_predict(X_scaled)

# Adicione os rótulos dos clusters ao DataFrame original
sampled_data['Cluster_Labels'] = labels

# Visualize os clusters
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels, cmap='viridis')
plt.xlabel('Feature 1 (Carteira_Servicos)')
plt.ylabel('Feature 2 (Carteira_Comercial)')
plt.title('DBSCAN Clusters')
plt.show()

#### Mean Shift

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
google_id = '1teoxduT3bJAkrD0k1YIPn-mbVI83oQox'
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id=google_id,
                                    dest_path = './dados_cvm.csv',
                                    showsize = True)
df_main = pd.read_csv('dados_cvm.csv')

google_id = '1J0fF-82tDg70wjaNg2Q8OGxPikA7phqp'
gdd.download_file_from_google_drive(file_id=google_id,
                                    dest_path = './dados_classes.csv',
                                    showsize = True)
df_classes = pd.read_csv('dados_classes.csv')

In [ ]:
# Drop rows with missing values in the "Patrimonio_Liquido" column and store the cleaned DataFrame in df_cleaned.
df_cleaned = df_main.dropna(subset=["Patrimonio_Liquido"])

# Extract the "Carteira_Direitos_Aquisicao_Inadimplentes" column from the cleaned DataFrame.
inadimplentes = df_cleaned["Carteira_Direitos_Aquisicao_Inadimplentes"]

# Extract the "Patrimonio_Liquido" column from the cleaned DataFrame.
patrimonio_liquido = df_cleaned["Patrimonio_Liquido"]

# Calculate the default rate by dividing the number of inadimplentes by the patrimonio_liquido.
taxa_inadimplencia = inadimplentes / patrimonio_liquido

# Display the calculated default rate.
taxa_inadimplencia

# Add a new column named "taxa_inadimplencia_series" to the original df_informe_mensal DataFrame
# and assign it the values of the calculated default rate.
df_main = df_main.assign(taxa_inadimplencia_series=taxa_inadimplencia)

In [ ]:
fundos_inadimplentes = df_main[
    (df_main['taxa_inadimplencia_series'] != 0) &
    df_main['taxa_inadimplencia_series'].notna()
]

list(fundos_inadimplentes['Nome_Fundo'].unique())

In [ ]:
carteiras = [
    'Carteira_Comercial',
    'Carteira_Comercial_Varejo',
]

summed_data = df_main.groupby('taxa_inadimplencia_series')[carteiras].sum()
plt.figure(figsize=(14, 8))
ax = sns.barplot(data=summed_data, palette='viridis')
plt.title('Sum of Default Rates by Portfolio')
plt.xlabel('Total Default Rate')
plt.ylabel('Portfolio')
plt.xticks(rotation=45, ha='right')

plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
summed_data.head(5)
X = summed_data

#### CARTEIRA COMERCIAL

In [ ]:
from sklearn.cluster import MeanShift, estimate_bandwidth

bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)

# Criar uma instância do MeanShift com a largura de banda estimada
ms = MeanShift(bandwidth=bandwidth)

# Ajustar o modelo aos dados
ms.fit(X)

# Obter os rótulos dos clusters e os centros dos clusters
labels = ms.labels_
cluster_centers = ms.cluster_centers_

# Plotar os pontos de dados e os centros dos clusters
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=labels)
plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1], s=300, linewidths=3, color='r')
plt.show()

In [ ]:
summed_data["Cluster_Labels"] = labels
summed_data

In [ ]:
dict(summed_data["Cluster_Labels"].value_counts())

In [ ]:
def selecionar_cluster(n_cluster):
  df_cluster = summed_data.loc[summed_data["Cluster_Labels"]==n_cluster]
  return df_cluster

In [ ]:
selecionar_cluster(0) #Adicionar o número de cluster que você gostaria de verificar

In [ ]:
import matplotlib.pyplot as plt

# Filtrar os dados para manter apenas os valores no intervalo de 0 a 5
filtered_data = summed_data[(summed_data["Cluster_Labels"] >= 0) & (summed_data["Cluster_Labels"] <= 5)]

# Contar a ocorrência de cada valor e criar o gráfico de barras
value_counts = filtered_data["Cluster_Labels"].value_counts()
value_counts = value_counts.sort_index()  # Ordenar pelo valor do índice

# Plotar o gráfico de barras
value_counts.plot(kind='bar')
plt.title('Cluster Labels no intervalo de 0 a 5')
plt.xlabel('Cluster Labels')
plt.ylabel('Contagem')
plt.show()

In [ ]:
df_cluster_s_0 = summed_data.loc[summed_data["Cluster_Labels"]!=0]

Temos algumas conclusões com esse gráfico.

1. Os dados estão extremamente espaçados e dispersos;

2. Maior concentração de números zerados estão no cluster 0;

3. O método de MeanShift pode não ser o melhor para os dados, pois ele busca a média dos clusters de acordo com uma função de densidade. No nosso caso, foi o 0.

# Escolha do modelo

Ao analisar a separação dos clusters da carteira comercial entre os algoritmos, o modelo escolhido foi o K-Means, pelo fato da sua separação ter sido uma das mais lógicas em relação ao nível de inadimplência, apesar do modelo não classificar os fundos relacionando-os diretamente com as features. Logo, é possível concluir que nenhum dos modelos candidatos apresentou uma separação muito clara ou significativa entre os clusters. Enquanto o K-Means e o Mini Batch K-Means realizaram uma separação em 3 partes proporcionais em relação ao target, o Mean Shift e o DBSCAN acabaram formando um cluster muito grande por conta de uma densidade de dados muito alta em determina, resultando em uma divisão muito desigual e também pouco significativa.